In [1]:
import numpy as np
import matplotlib.pyplot as plt
from utilities import load_data

X_train,y_train,X_test,y_test = load_data()
print(X_train.sahpe)
print(y_train.sahpe)

ModuleNotFoundError: No module named 'h5py'